# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114539e+02     1.430587e+00
 * time: 0.5247831344604492
     1     1.024311e+01     8.334153e-01
 * time: 1.5216341018676758
     2    -1.254519e+01     9.970650e-01
 * time: 2.002624988555908
     3    -3.430072e+01     7.073248e-01
 * time: 2.7376530170440674
     4    -4.772519e+01     4.926740e-01
 * time: 3.4394190311431885
     5    -5.696139e+01     1.810241e-01
 * time: 4.159497022628784
     6    -5.979199e+01     1.216476e-01
 * time: 4.6528401374816895
     7    -6.087323e+01     5.520677e-02
 * time: 5.125286102294922
     8    -6.131904e+01     6.750431e-02
 * time: 5.5990049839019775
     9    -6.159822e+01     3.718037e-02
 * time: 6.0738091468811035
    10    -6.182628e+01     2.940485e-02
 * time: 6.564428091049194
    11    -6.198799e+01     1.887401e-02
 * time: 7.058311939239502
    12    -6.205550e+01     1.520367e-02
 * time: 7.528166055679321
    13    -6.211748e+01     1.482379e-02
 * time: 8.00123906135559
 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671077
    AtomicLocal         -18.8557700
    AtomicNonlocal      14.8522652
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336824

    total               -62.261666452272
